In [1]:
import json
import csv
import sys
import pandas as pd
from collections import defaultdict

csv.field_size_limit(sys.maxsize)

all_ratings = pd.read_csv('./work/data/collected_animelists.csv')

class RatingSumCounter:
    sum = 0
    count = 0

# Compute total + average ratings per anime
rating_sums_by_anime = defaultdict(lambda: RatingSumCounter())

i = 0
for row in all_ratings.itertuples():
    anime_id = row.anime_id
    rating = row.my_score
    ratings = rating_sums_by_anime[anime_id]
    ratings.sum += rating
    ratings.count += 1

    i += 1
    if i % 1000000 == 0:
        print(f"Processed {i} ratings")

# exclude all anime that have a rating of "rx" in the metadata
excluded_ids = set()

# Exported from MySQL
with open('./work/data/anime-metadata.csv') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    # skip header row
    next(reader)

    # Read by the webapp backend
    with open('./work/data/processed-metadata.csv', 'wt') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"')
        headers = ['id', 'title', 'title_english', 'related_anime', 'recommendations', 'aired_from_year', 'rating_count', 'average_rating', 'media_type', 'rating']
        writer.writerow(headers)

        for row in reader:
            anime_id = int(row[0])
            rating_count = rating_sums_by_anime[anime_id].count
            if rating_count < 100:
                continue
            metadata = json.loads(row[1])
            if metadata is None:
                print(f"No metadata for anime {anime_id}")
                continue
            title = metadata['title']
            title_english = metadata['alternative_titles']['en'] if 'en' in metadata['alternative_titles'] else title
            related_anime = json.dumps(metadata.get('related_anime', []))
            recommendations = json.dumps(metadata.get('recommendations', []))
            aired_from_year = int(metadata.get('start_date', '1900')[:4])
            average_rating = rating_sums_by_anime[anime_id].sum / rating_count if rating_count > 0 else 0
            media_type = metadata.get('media_type') or 'unknown'
            if metadata.get('media_type') is None:
                print(f"No media type for {anime_id}")
            rating = metadata.get('rating') or 'Unknown'
            if rating.lower() == 'rx':
                excluded_ids.add(anime_id)
            writer.writerow([anime_id, title, title_english, related_anime, recommendations, aired_from_year, rating_count, average_rating, media_type, rating])

Processed 1000000 ratings
Processed 2000000 ratings
Processed 3000000 ratings
Processed 4000000 ratings
Processed 5000000 ratings
Processed 6000000 ratings
Processed 7000000 ratings
Processed 8000000 ratings
Processed 9000000 ratings
Processed 10000000 ratings
Processed 11000000 ratings
Processed 12000000 ratings
Processed 13000000 ratings
Processed 14000000 ratings
Processed 15000000 ratings
Processed 16000000 ratings
Processed 17000000 ratings
Processed 18000000 ratings
Processed 19000000 ratings
Processed 20000000 ratings
Processed 21000000 ratings
Processed 22000000 ratings
Processed 23000000 ratings
Processed 24000000 ratings
Processed 25000000 ratings
Processed 26000000 ratings
Processed 27000000 ratings
Processed 28000000 ratings
Processed 29000000 ratings
Processed 30000000 ratings
Processed 31000000 ratings
Processed 32000000 ratings
Processed 33000000 ratings
Processed 34000000 ratings
Processed 35000000 ratings
Processed 36000000 ratings
Processed 37000000 ratings
Processed 

In [2]:
import json

corpus_size = 6000
# pick top N by rating count
outfile_name = './work/data/corpus_ids.json'

with open(outfile_name, 'wt') as f:
    corpus_ids = sorted(rating_sums_by_anime.keys(), key=lambda anime_id: rating_sums_by_anime[anime_id].count, reverse=True)
    corpus_ids = [anime_id for anime_id in corpus_ids if anime_id not in excluded_ids]
    corpus_ids = corpus_ids[:corpus_size]
    json.dump(corpus_ids, f)

In [3]:
print(f"Excluded {len(excluded_ids)} anime from corpus due to 'rx' rating")

Excluded 1597 anime from corpus due to 'rx' rating
